In [2]:
!pip install librosa fpdf
 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40769 sha256=c2b0aad820ff922c9fcd07f1e070347fc43900af5a19ce3ec888c60c4b2c0d42
  Stored in directory: c:\users\sagni\appdata\local\pip\cache\wheels\65\4f\66\bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


  DEPRECATION: Building 'fpdf' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fpdf'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [4]:
pip install sounddevice scipy librosa fpdf


Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import sounddevice as sd
from scipy.io.wavfile import write
import librosa
import numpy as np
from fpdf import FPDF
from datetime import datetime

# ----------- Record Audio from Microphone -----------
def record_audio(filename, duration=5, samplerate=44100):
    print("🎙️ Recording started... Speak now.")
    recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()
    write(filename, samplerate, recording)
    print(f"✅ Recording saved as {filename}")

# ----------- Analyze Voice Tone -----------
def analyze_voice_tone(audio_path):
    y, sr = librosa.load(audio_path)

    pitch = librosa.yin(y, fmin=50, fmax=300)
    energy = np.mean(librosa.feature.rms(y=y))

    avg_pitch = np.mean(pitch)

    if avg_pitch > 200 and energy > 0.05:
        tone = "Excited or Anxious"
    elif avg_pitch < 130 and energy < 0.02:
        tone = "Calm or Sad"
    else:
        tone = "Neutral"

    return {
        "Average Pitch": float(round(avg_pitch, 2)),
        "Energy": float(round(energy, 4)),
        "Tone": tone
    }

# ----------- Generate PDF Report -----------
def generate_pdf_report(face_emotion, tone_analysis, output_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, "Emotion-Aware Virtual Interview Report", ln=True, align='C')

    pdf.set_font("Arial", size=12)
    pdf.ln(10)
    pdf.cell(200, 10, f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)
    
    pdf.ln(10)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(200, 10, "Facial Expression Analysis:", ln=True)
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, f"Detected Emotion: {face_emotion}", ln=True)

    pdf.ln(10)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(200, 10, "Voice Tone Analysis:", ln=True)
    pdf.set_font("Arial", size=12)
    for key, val in tone_analysis.items():
        pdf.cell(200, 10, f"{key}: {val}", ln=True)

    pdf.output(output_path)
    print(f"📄 PDF report saved to: {output_path}")

# ----------- Paths and Execution -----------
folder = r"C:\Users\sagni\Downloads\Emotion Aware Virtual Interviewer"
os.makedirs(folder, exist_ok=True)

audio_file = os.path.join(folder, "interview_audio.wav")
pdf_file = os.path.join(folder, "interview_report.pdf")

record_audio(audio_file, duration=6)  # record for 6 seconds

tone_result = analyze_voice_tone(audio_file)

face_emotion = input("🤖 Enter detected facial emotion (e.g., Happy, Nervous, Sad): ")

generate_pdf_report(face_emotion, tone_result, pdf_file)


🎙️ Recording started... Speak now.
✅ Recording saved as C:\Users\sagni\Downloads\Emotion Aware Virtual Interviewer\interview_audio.wav


🤖 Enter detected facial emotion (e.g., Happy, Nervous, Sad):  happy


📄 PDF report saved to: C:\Users\sagni\Downloads\Emotion Aware Virtual Interviewer\interview_report.pdf
